# 如何创建自定义文档加载器
## 概述

基于大型语言模型（LLM）的应用通常需要从数据库或文件（如PDF）中提取数据，并将其转换为LLM可用的格式。在LangChain中，这一过程通常涉及创建文档对象（Document），这些对象封装了提取的文本（`page_content`）以及元数据——一个包含文档相关细节（如作者姓名或发布日期）的字典。
`Document` 对象通常会被格式化为提示词，输入到大型语言模型（LLM）中，使 LLM 能够利用 `Document` 中的信息生成所需的响应（例如总结文档内容）。`文档`可以立即使用，也可以索引到向量存储中，以便未来检索和使用。
[文档加载](/docs/concepts/document_loaders/)的主要抽象概念包括：

| 组件          | 描述                    ||----------------|--------------------------------|| 文档         | 包含 `文本` 和 `元数据` || BaseLoader     | 用于将原始数据转换为 `Documents`  || Blob           | 表示位于文件或内存中的二进制数据 || BaseBlobParser | 用于解析 `Blob` 以生成 `Document` 对象的逻辑 |
本指南将演示如何编写自定义文档加载与文件解析逻辑，具体内容包括：
1. 通过从 `BaseLoader` 子类化创建一个标准文档加载器。2. 使用 `BaseBlobParser` 创建一个解析器，并将其与 `Blob` 和 `BlobLoaders` 结合使用。这种方法主要适用于处理文件时。

## 标准文档加载器
文档加载器可以通过继承 `BaseLoader` 来实现，该基类为加载文档提供了标准接口。
### 接口
| 方法名称 | 说明 ||-------------|-------------|| lazy_load   | 用于**懒加载**文档，逐个加载。适用于生产环境代码。 || alazy_load  | `lazy_load` 的异步变体 || 加载        | 用于**立即**将所有文档加载到内存中。适用于原型设计或交互式工作。 || aload       | 用于**主动**将所有文档加载到内存中。适用于原型设计或交互式工作场景。**该功能于2024年4月加入LangChain。** |
* `load` 方法是一个仅用于原型设计的便捷方法——它只是调用了 `list(self.lazy_load())`。* `alazy_load` 有一个默认实现，该实现会委托给 `lazy_load`。如果您正在使用异步操作，我们建议覆盖默认实现并提供一个原生的异步实现。
:::重要在实现文档加载器时，**切勿**通过 `lazy_load` 或 `alazy_load` 方法提供参数。
所有配置都应通过初始化方法（`__init__`）传递。这是LangChain的一项设计决策，旨在确保文档加载器一旦实例化，便具备加载文档所需的全部信息。:::

### 实施
让我们创建一个标准文档加载器的示例，该加载器会读取文件并根据文件中的每一行生成一个文档。

In [1]:
from typing import AsyncIterator, Iterator

from langchain_core.document_loaders import BaseLoader
from langchain_core.documents import Document


class CustomDocumentLoader(BaseLoader):
    """An example document loader that reads a file line by line."""

    def __init__(self, file_path: str) -> None:
        """Initialize the loader with a file path.

        Args:
            file_path: The path to the file to load.
        """
        self.file_path = file_path

    def lazy_load(self) -> Iterator[Document]:  # <-- Does not take any arguments
        """A lazy loader that reads a file line by line.

        When you're implementing lazy load methods, you should use a generator
        to yield documents one by one.
        """
        with open(self.file_path, encoding="utf-8") as f:
            line_number = 0
            for line in f:
                yield Document(
                    page_content=line,
                    metadata={"line_number": line_number, "source": self.file_path},
                )
                line_number += 1

    # alazy_load is OPTIONAL.
    # If you leave out the implementation, a default implementation which delegates to lazy_load will be used!
    async def alazy_load(
        self,
    ) -> AsyncIterator[Document]:  # <-- Does not take any arguments
        """An async lazy loader that reads a file line by line."""
        # Requires aiofiles (install with pip)
        # https://github.com/Tinche/aiofiles
        import aiofiles

        async with aiofiles.open(self.file_path, encoding="utf-8") as f:
            line_number = 0
            async for line in f:
                yield Document(
                    page_content=line,
                    metadata={"line_number": line_number, "source": self.file_path},
                )
                line_number += 1

### 测试 🧪

为了测试文档加载器，我们需要一个包含高质量内容的文件。

In [2]:
with open("./meow.txt", "w", encoding="utf-8") as f:
    quality_content = "meow meow🐱 \n meow meow🐱 \n meow😻😻"
    f.write(quality_content)

loader = CustomDocumentLoader("./meow.txt")

In [3]:
## Test out the lazy load interface
for doc in loader.lazy_load():
    print()
    print(type(doc))
    print(doc)


<class 'langchain_core.documents.base.Document'>
page_content='meow meow🐱 \n' metadata={'line_number': 0, 'source': './meow.txt'}

<class 'langchain_core.documents.base.Document'>
page_content=' meow meow🐱 \n' metadata={'line_number': 1, 'source': './meow.txt'}

<class 'langchain_core.documents.base.Document'>
page_content=' meow😻😻' metadata={'line_number': 2, 'source': './meow.txt'}


In [4]:
## Test out the async implementation
async for doc in loader.alazy_load():
    print()
    print(type(doc))
    print(doc)


<class 'langchain_core.documents.base.Document'>
page_content='meow meow🐱 \n' metadata={'line_number': 0, 'source': './meow.txt'}

<class 'langchain_core.documents.base.Document'>
page_content=' meow meow🐱 \n' metadata={'line_number': 1, 'source': './meow.txt'}

<class 'langchain_core.documents.base.Document'>
page_content=' meow😻😻' metadata={'line_number': 2, 'source': './meow.txt'}


:::提示
`load()` 在交互式环境（例如 Jupyter notebook）中非常实用。
避免在生产代码中使用它，因为预加载假设所有内容可以放入内存，但这种情况并不常见，尤其是对于企业数据而言。:::

In [6]:
loader.load()

[Document(page_content='meow meow🐱 \n', metadata={'line_number': 0, 'source': './meow.txt'}),
 Document(page_content=' meow meow🐱 \n', metadata={'line_number': 1, 'source': './meow.txt'}),
 Document(page_content=' meow😻😻', metadata={'line_number': 2, 'source': './meow.txt'})]

## 文件操作
许多文档加载器都涉及文件解析。这些加载器之间的差异通常源于文件的解析方式，而非文件的加载方式。例如，你可以使用 `open` 来读取 PDF 或 Markdown 文件的二进制内容，但需要不同的解析逻辑将这些二进制数据转换为文本。
因此，将解析逻辑与加载逻辑解耦会很有帮助，这样无论数据如何加载，都能更容易地复用给定的解析器。
### BaseBlobParser
`BaseBlobParser` 是一个接口，它接收一个 `blob` 并输出一个 `Document` 对象列表。`blob` 是存在于内存或文件中的数据表示形式。LangChain Python 中的 `Blob` 原语灵感来源于 [Blob WebAPI 规范](https://developer.mozilla.org/en-US/docs/Web/API/Blob)。

In [7]:
from langchain_core.document_loaders import BaseBlobParser, Blob


class MyParser(BaseBlobParser):
    """A simple parser that creates a document from each line."""

    def lazy_parse(self, blob: Blob) -> Iterator[Document]:
        """Parse a blob into a document line by line."""
        line_number = 0
        with blob.as_bytes_io() as f:
            for line in f:
                line_number += 1
                yield Document(
                    page_content=line,
                    metadata={"line_number": line_number, "source": blob.source},
                )

In [8]:
blob = Blob.from_path("./meow.txt")
parser = MyParser()

In [8]:
list(parser.lazy_parse(blob))

[Document(page_content='meow meow🐱 \n', metadata={'line_number': 1, 'source': './meow.txt'}),
 Document(page_content=' meow meow🐱 \n', metadata={'line_number': 2, 'source': './meow.txt'}),
 Document(page_content=' meow😻😻', metadata={'line_number': 3, 'source': './meow.txt'})]

使用 **blob** API 还能直接从内存加载内容，无需从文件中读取！

In [9]:
blob = Blob(data=b"some data from memory\nmeow")
list(parser.lazy_parse(blob))

[Document(page_content='some data from memory\n', metadata={'line_number': 1, 'source': None}),
 Document(page_content='meow', metadata={'line_number': 2, 'source': None})]

### Blob
让我们快速浏览一下Blob API的部分内容。

In [10]:
blob = Blob.from_path("./meow.txt", metadata={"foo": "bar"})

In [11]:
blob.encoding

'utf-8'

In [12]:
blob.as_bytes()

b'meow meow\xf0\x9f\x90\xb1 \n meow meow\xf0\x9f\x90\xb1 \n meow\xf0\x9f\x98\xbb\xf0\x9f\x98\xbb'

In [13]:
blob.as_string()

'meow meow🐱 \n meow meow🐱 \n meow😻😻'

In [14]:
blob.as_bytes_io()

In [15]:
blob.metadata

{'foo': 'bar'}

In [16]:
blob.source

'./meow.txt'

### Blob 加载器
虽然解析器封装了将二进制数据解析为文档所需的逻辑，但**blob加载器**则封装了从给定存储位置加载blob所需的逻辑。
目前，`LangChain` 仅支持 `FileSystemBlobLoader`。
您可以使用 `FileSystemBlobLoader` 加载 blob，然后使用解析器对其进行解析。

In [17]:
from langchain_community.document_loaders.blob_loaders import FileSystemBlobLoader

blob_loader = FileSystemBlobLoader(path=".", glob="*.mdx", show_progress=True)

In [18]:
parser = MyParser()
for blob in blob_loader.yield_blobs():
    for doc in parser.lazy_parse(blob):
        print(doc)
        break

  0%|          | 0/8 [00:00<?, ?it/s]

page_content='# Microsoft Office\n' metadata={'line_number': 1, 'source': 'office_file.mdx'}
page_content='# Markdown\n' metadata={'line_number': 1, 'source': 'markdown.mdx'}
page_content='# JSON\n' metadata={'line_number': 1, 'source': 'json.mdx'}
page_content='---\n' metadata={'line_number': 1, 'source': 'pdf.mdx'}
page_content='---\n' metadata={'line_number': 1, 'source': 'index.mdx'}
page_content='# File Directory\n' metadata={'line_number': 1, 'source': 'file_directory.mdx'}
page_content='# CSV\n' metadata={'line_number': 1, 'source': 'csv.mdx'}
page_content='# HTML\n' metadata={'line_number': 1, 'source': 'html.mdx'}


### 通用加载器
LangChain 提供了一个名为 `GenericLoader` 的抽象类，它由 `BlobLoader` 和 `BaseBlobParser` 组合而成。
`GenericLoader` 旨在提供标准化的类方法，以便轻松使用现有的 `BlobLoader` 实现。目前仅支持 `FileSystemBlobLoader`。

In [19]:
from langchain_community.document_loaders.generic import GenericLoader

loader = GenericLoader.from_filesystem(
    path=".", glob="*.mdx", show_progress=True, parser=MyParser()
)

for idx, doc in enumerate(loader.lazy_load()):
    if idx < 5:
        print(doc)

print("... output truncated for demo purposes")

  0%|          | 0/8 [00:00<?, ?it/s]

page_content='# Microsoft Office\n' metadata={'line_number': 1, 'source': 'office_file.mdx'}
page_content='\n' metadata={'line_number': 2, 'source': 'office_file.mdx'}
page_content='>[The Microsoft Office](https://www.office.com/) suite of productivity software includes Microsoft Word, Microsoft Excel, Microsoft PowerPoint, Microsoft Outlook, and Microsoft OneNote. It is available for Microsoft Windows and macOS operating systems. It is also available on Android and iOS.\n' metadata={'line_number': 3, 'source': 'office_file.mdx'}
page_content='\n' metadata={'line_number': 4, 'source': 'office_file.mdx'}
page_content='This covers how to load commonly used file formats including `DOCX`, `XLSX` and `PPTX` documents into a document format that we can use downstream.\n' metadata={'line_number': 5, 'source': 'office_file.mdx'}
... output truncated for demo purposes


#### 自定义通用加载器
如果你真的喜欢创建类，可以通过子类化并创建一个类来将逻辑封装在一起。
你可以通过继承这个类，使用现有的加载器来加载内容。

In [20]:
from typing import Any


class MyCustomLoader(GenericLoader):
    @staticmethod
    def get_parser(**kwargs: Any) -> BaseBlobParser:
        """Override this method to associate a default parser with the class."""
        return MyParser()

In [21]:
loader = MyCustomLoader.from_filesystem(path=".", glob="*.mdx", show_progress=True)

for idx, doc in enumerate(loader.lazy_load()):
    if idx < 5:
        print(doc)

print("... output truncated for demo purposes")

  0%|          | 0/8 [00:00<?, ?it/s]

page_content='# Microsoft Office\n' metadata={'line_number': 1, 'source': 'office_file.mdx'}
page_content='\n' metadata={'line_number': 2, 'source': 'office_file.mdx'}
page_content='>[The Microsoft Office](https://www.office.com/) suite of productivity software includes Microsoft Word, Microsoft Excel, Microsoft PowerPoint, Microsoft Outlook, and Microsoft OneNote. It is available for Microsoft Windows and macOS operating systems. It is also available on Android and iOS.\n' metadata={'line_number': 3, 'source': 'office_file.mdx'}
page_content='\n' metadata={'line_number': 4, 'source': 'office_file.mdx'}
page_content='This covers how to load commonly used file formats including `DOCX`, `XLSX` and `PPTX` documents into a document format that we can use downstream.\n' metadata={'line_number': 5, 'source': 'office_file.mdx'}
... output truncated for demo purposes
